# Multiple Results

In this notebook we show how you can in a single request, have the LLM model return multiple results per prompt. This is useful for running experiments where you want to evaluate the robustness of your prompt and the parameters of your config against a particular large language model.

In [ ]:
!python -m pip install semantic-kernel==0.4.3.dev0

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai import ChatRequestSettings, CompleteRequestSettings
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, AzureChatCompletion, OpenAITextCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

First, we will set up the text and chat services we will be submitting prompts to.

In [ ]:
kernel = sk.Kernel()

# Configure Azure LLM service
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
azure_text_service = AzureTextCompletion(deployment_name="text", endpoint=endpoint, api_key=api_key)    # set the deployment name to the value of your text model
azure_chat_service = AzureChatCompletion(deployment_name="turbo", endpoint=endpoint, api_key=api_key)   # set the deployment name to the value of your chat model


# Configure OpenAI service
api_key, org_id = sk.openai_settings_from_dot_env()
oai_text_service = OpenAITextCompletion(ai_model_id="text-davinci-003", api_key=api_key, org_id=org_id)
oai_chat_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)

# Configure Hugging Face service
hf_text_service = HuggingFaceTextCompletion(ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.

In [ ]:
request_settings = CompleteRequestSettings(
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3
)

## Multiple Open AI Text Completions

In [ ]:
prompt = "what is the purpose of a rubber duck?"
results = await oai_text_service.complete_async(prompt=prompt, settings=request_settings)
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

## Multiple Azure Open AI Text Completions

In [ ]:
prompt = "provide me a list of possible meanings for the acronym 'ORLD'"
results = await azure_text_service.complete_async(prompt=prompt, settings=request_settings)
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

## Multiple Hugging Face Text Completions

In [ ]:
prompt = "The purpose of a rubber duck is"
results = await hf_text_service.complete_async(prompt=prompt, request_settings=request_settings)
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

Here, we're setting up the settings for Chat completions.

In [ ]:
chat_request_settings = ChatRequestSettings(
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3
)

## Multiple OpenAI Chat Completions

In [ ]:
role = "user"
content = "It's a beautiful day outside, birds are singing, flowers are blooming. On days like these, kids like you..."
message = { "role":role, "content":content }
results = await oai_chat_service.complete_chat_async(messages=[message], settings=chat_request_settings)
i = 0
for result in results:
    print(f"Result {i}: {result}")
    i += 1

## Multiple Azure OpenAI Chat Completions

In [ ]:
role = "user"
content = "Tomorow is going to be a great day, I can feel it. I'm going to wake up early, go for a run, and then..."
message = { "role":role, "content":content }
results = await azure_chat_service.complete_chat_async(messages=[message], settings=chat_request_settings)
i = 0
for result in results:
    print(f"Result {i}: {result}")
    i += 1

## Streaming Multiple Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.

In [ ]:
import os
from IPython.display import clear_output

if os.name == "nt":
    clear = "cls"
else:
    clear = "clear"

prompt = "what is the purpose of a rubber duck?"
stream = oai_text_service.complete_stream_async(prompt=prompt, settings=request_settings)
texts = [""] * request_settings.number_of_responses
async for results in stream:
    i = 1
    clear_output(wait=True)
    for result in results:
        texts[i - 1] += result
        print(f"Result {i}: {texts[i - 1]}")
        i += 1

print("----------------------------------------")
